# 4 - IP Build RCAs from Pre-Processed NetMap Reaches and DEM

Use this tool to build reach contributing area (RCAs) from a NetMap stream reaches and a digital elevation model (DEM). This tool is designed to replace the STARS "Generate Cost RCAs" tool which was built using Python 2 and designed for use in ArcMap.

## Required Software:

- The code contained in this notebook is designed to be run within an ESRI ArcPro project. The script output is written to the default project geodatabase and therefore the user must open and run the notebook .IPYNB file within an ArcPro project.
- If any of the geoprocessing steps require an advanced license or any specific extensions, the script will check for these conditions before running.
- A modified version of this script has been included as a .PY file, and added to the **IP.tbx** ESRI toolbox included with this package. The setup code block in that version of the script has been modified to accept input parameters from the ArcPro GUI instead of notebook code. That alternate setup code block is included in this script for reference, but has been hashed out.

## Required Inputs:

- A DEM, preferably the same source that was used to create the NetMap reaches. For the Yukon-Kuskokwim IP project, this is the 5m IFSAR Digital Surface Model (DSM) that can be downloaded from various repositories online. See the Data.gov catalog entry for metadata (https://catalog.data.gov/dataset/5-meter-alaska-digital-elevation-models-dems-usgs-national-map-3dep-downloadable-data-collectio) and the USGS National Map online application for a download option (https://apps.nationalmap.gov/downloader/).

- NetMap synthetic stream reaches, clipped to HUC8 extent and pruned to keep only those reaches with greater than 5km catchment area. These reaches should be pre-processed using the "Prune Netmap Reaches to 5km and Clip by HUC8" script included in this toolbox. (Attempting to process a full size NetMap product with a 5m DEM will likely crash the program.)

- A unique attribute field in the pre-processed NetMap synthetic stream reach feature layer. The values in this field will become the unique RCA IDs.

- A feature class name for the RCA polygon output.

- A system directory to use as a temporary file location. It is recommended that this be a short file path on an internal hard drive (eg, "C:/data").


## Geoprocessing Output:

- An RCA polygon feature class written to the default project geodatabase.

## Processing Steps:

1. Convert the pre-processed reaches to a raster, using the same cell size and extent as the input DEM.
2. Create a raster object from the rasterized streamlines.
3. Run a series of Raster Calculator expressions. (These calculations essentially create a cost surface where ridgeline landscape positions become very expensive compared to valley and hillslope positions. This will constrain the RCA polygons and prevent them from crossing ridgetops into adjacent drainages.)
4. Run the Cost Allocation using the rasterized stream reaches and the final cost surface.
5. Build a raster attribute table for the the cost allocation output.
6. Convert the cost allocation output to polygon, and dissolve any multi-part polygons.
7. Add a field for the RCA ID, then calculate the RCA ID to equal the "gridcode" attribute from the original rasterized reaches.
8. Delete all intermediary rasters.

### Code starts here:

#### Setup

Import modules and reset environments to default. This should set the ArcPro project geodatabase as the workspace/scratch environment, just in case it was set otherwise. Additionally, prevent the addition of intermediary outputs to the ArcPro project map. Some tools may not run if their target is open in the map display.

In [4]:
import os, arcpy, sys,datetime, traceback

import arcpy.management

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
print(f'Date: {datetime.datetime.now()}')
print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = r"D:\\GIS\\AKSSF_ValBot_2023"
outgdb = os.path.join(outdir,'AKSSF_ValBot.gdb')
print(f'Output directory set to {outdir}')
print(f'Output gdb set to {outgdb}')

Date: 2023-11-14 10:07:55.919164
imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology\\confinement_scripts', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python39.zip', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\P

The Spatial Analyst extension is required for this script to work. Check for extension, and if its available check it out for use. If it is unavailable or cannot be checked out, throw a license error and provide an error message.

In [2]:
print("Spatial Analyst license is required.")
arcpy.AddMessage("Spatial Analyst license is required.")
print(arcpy.GetMessages())

class LicenseError(Exception):
    pass

try:
    if arcpy.CheckExtension("Spatial") == "Available":
        arcpy.CheckOutExtension("Spatial")
        print("Good news...Spatial Analyst license is available!")
        arcpy.AddMessage("Good news...Spatial Analyst license is available!")
        print(arcpy.GetMessages())
    else:
        # raise a custom exception
        raise LicenseError

except LicenseError:
    print("Bad news...Spatial Analyst license is unavailable.")
    arcpy.AddMessage("Bad news...Spatial Analyst license is unavailable.")
    print(arcpy.GetMessages())
except arcpy.ExecuteError:
    print(arcpy.GetMessages())

Spatial Analyst license is required.

Good news...Spatial Analyst license is available!


In [3]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

In [4]:
import time
import datetime
import os
import arcpy

prefDict = {'Bristol_Bay':'bb', 'Kodiak':'kod', 'Prince_William_Sound':'pws','Cook_Inlet':'ci','Copper_River':'cr'}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet', 'Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Set input regional folders
regions = [f'D:\\GIS\\AKSSF\\{source}' for source in nhdplus_dat + tauDem_dat]

strDict = {}
roiRasDict = {}

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
print(f'Begin {datetime.datetime.now()}')

for region in regions:
    roi = os.path.basename(region)
    print(roi)

    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    print(f'GDB {gdb}')

    walk = arcpy.da.Walk(region, datatype=['FeatureClass', 'RasterDataset'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if filename == 'NHDFlowline_merge' and roi in nhdplus_dat:
                streamname = f'{roi}_{filename}'
                streams = os.path.join(outgdb, streamname)
                if not arcpy.Exists(streams):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outgdb}')
                    startTime = time.time()  # Start timer
                    # Select only streams/rivers and artificial paths
                    arcpy.FeatureClassToFeatureClass_conversion(
                        in_features=os.path.join(dirpath, filename),
                        out_path=outgdb,
                        out_name=streamname,
                        where_clause="FType = 460 Or FType = 558"
                    )
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{streams} already created')
                append_value(strDict, roi, [streams, 'NHDPlusID'])

            elif filename == 'streams_merge' and roi in tauDem_dat:
                if roi == 'Bristol_Bay':
                    id = 'catID'
                else:
                    id = 'LINKNO'

                streamname = f'{roi}_{filename}'
                streams = os.path.join(outgdb, streamname)
                if not arcpy.Exists(streams):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outgdb}')
                    startTime = time.time()  # Start timer
                    arcpy.FeatureClassToFeatureClass_conversion(
                        os.path.join(dirpath, filename),
                        outgdb,
                        streamname
                    )
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{streams} already created')
                append_value(strDict, roi, [streams, id])

            elif filename.endswith('.tif'):
                if filename == 'elev.tif':
                    raster_type = 'elev'
                elif filename == 'slope.tif':
                    raster_type = 'slope'
                elif filename == 'fac.tif':
                    raster_type = 'fac'
                elif filename == 'fdr.tif':
                    raster_type = 'fdr'
                else:
                    continue  # Skip if not the required raster

                raster_name = f'{prefDict[roi]}{raster_type}.tif'
                raster_path = os.path.join(outdir, raster_name)
                if not arcpy.Exists(raster_path):
                    print(f'Copying {os.path.join(dirpath, filename)} to {outdir}')
                    startTime = time.time()  # Start timer
                    arcpy.CopyRaster_management(os.path.join(dirpath, filename), raster_path)
                    endTime = time.time()  # End timer
                    elapsedTime = endTime - startTime  # Calculate elapsed time
                    print(f'Copy completed in {elapsedTime} seconds')
                else:
                    print(f'{raster_path} already created')
                append_value(roiRasDict, roi, [raster_path])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success
print(f'{"*"*100}')
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')
print(strDict)
print(f'{"*"*100}')
print(roiRasDict)

Begin 2023-11-14 08:18:47.211317
Cook_Inlet
GDB ['D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb']
D:\\GIS\\AKSSF_ValBot_2023\cielev.tif already created
D:\\GIS\\AKSSF_ValBot_2023\cifac.tif already created
D:\\GIS\\AKSSF_ValBot_2023\cifdr.tif already created
D:\\GIS\\AKSSF_ValBot_2023\cislope.tif already created
D:\\GIS\\AKSSF_ValBot_2023\AKSSF_ValBot.gdb\Cook_Inlet_NHDFlowline_merge already created
Copper_River
GDB ['D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb']
D:\\GIS\\AKSSF_ValBot_2023\crelev.tif already created
D:\\GIS\\AKSSF_ValBot_2023\crfac.tif already created
D:\\GIS\\AKSSF_ValBot_2023\crfdr.tif already created
D:\\GIS\\AKSSF_ValBot_2023\crslope.tif already created
D:\\GIS\\AKSSF_ValBot_2023\AKSSF_ValBot.gdb\Copper_River_NHDFlowline_merge already created
Bristol_Bay
GDB ['D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb']
D:\\GIS\\AKSSF_ValBot_2023\bbelev.tif already created
D:\\GIS\\AKSSF_ValBot_2023\bbfac.tif already created
D:\\GIS\\AKSSF_ValBot_2023\bbfdr.tif already created
D:

Set processing regions and paths to stream/flowline and elevation rasters.  Copy to outdir/gdb


In [5]:
from collections import Counter
combinedDict = {}

# Add key-value pairs from strDict to combinedDict
combinedDict = {x: roiRasDict.get(x, 0) + strDict.get(x, 0)
                    for x in set(roiRasDict).union(strDict)}

combinedDict



{'Copper_River': ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\crelev.tif',
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\crfac.tif'],
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\crfdr.tif'],
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\crslope.tif'],
  'D:\\\\GIS\\\\AKSSF_ValBot_2023\\AKSSF_ValBot.gdb\\Copper_River_NHDFlowline_merge',
  'NHDPlusID'],
 'Bristol_Bay': ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbelev.tif',
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbfac.tif'],
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbfdr.tif'],
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbslope.tif'],
  'D:\\\\GIS\\\\AKSSF_ValBot_2023\\AKSSF_ValBot.gdb\\Bristol_Bay_streams_merge',
  'catID'],
 'Kodiak': ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\kodelev.tif',
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\kodfac.tif'],
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\kodfdr.tif'],
  ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\kodslope.tif'],
  'D:\\\\GIS\\\\AKSSF_ValBot_2023\\AKSSF_ValBot.gdb\\Kodiak_streams_merge',
  'LINKNO'],
 'Prince_William_Sound': ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\pwselev.tif',
  ['D:\

Convert stream reaches to a raster, using the same cell size as the DEM. Also, use the DEM as the snap raster.

In [11]:
os.path.normpath(combinedDict['Kodiak'][0]) 


'D:\\GIS\\AKSSF_ValBot_2023\\kodelev.tif'

In [6]:
# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
print(f'Begin {datetime.datetime.now()}')

for k,v in combinedDict.items():
    if k == 'Bristol_Bay':
        roi = k
        reachRas = os.path.join(outdir,prefDict[k] + "rchRas.tif")
        # Start timing function
        processStart2 = time.time()
        processStartdt2 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')
        print(k,v)
        DEM_ras = arcpy.Raster(v[0])
        fac_ras = arcpy.Raster(v[1])
        fdr_ras = arcpy.Raster(v[2])
        slope_ras = arcpy.Raster(v[3])
        size = DEM_ras.meanCellWidth
        size = int(size)
        reachid = v[5]
        streams = v[4]
        arcpy.env.snapRaster = DEM_ras
        if not arcpy.Exists(reachRas):
            print(f'Converting {streams} to raster')
            arcpy.conversion.PolylineToRaster(streams,#streams
                                              reachid,#reachID
                                              reachRas,#reachRaster
                                              "",
                                              "",
                                              size,
                                              ""
                                              )
        else:
            print(f'{reachRas} already created')
        append_value(combinedDict,k,reachRas)
        # End timing
        processEnd2 = time.time()
        processElapsed2 = int(processEnd2 - processStart2)
        processSuccess_time2 = datetime.datetime.now()
        # Report success
        print(f'Stream to raster Process completed at {processSuccess_time2.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed2)})')
        print(f'{"*"*100}')


        # Focal Mean equation using the annulus shape around each raster cell. (Annulus size parameters used here are copied directly from the STARS tool.)
        # Start timing function
        processStart3 = time.time()
        processStartdt3 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')

        nbr = arcpy.sa.NbrAnnulus(7, 12, "CELL")
        Ann_Raw = arcpy.sa.Int((DEM_ras - (arcpy.sa.FocalStatistics(DEM_ras, nbr, "MEAN", ""))) + 0.5)

        # End timing
        processEnd3 = time.time()
        processElapsed3 = int(processEnd3 - processStart3)
        processSuccess_time3 = datetime.datetime.now()
        # Report success
        print(f'p3. Focal Mean Annulus Process completed at {processSuccess_time3.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed3)})')
        print(f'{"*"*100}')

        #Weight the ridgelines using a conditional statement. If the difference between the DEM elevation and the annulus focal mean was greater than 0, that means the pixel is probably on a convexity (ie, surrounded by lower elevation areas). In the convex condition, increase the elevation exponentionally to provide a heavy cost for traversing that pixel. In non-convex conditions, assign a value of 1.

        # Start timing function
        processStart4 = time.time()
        processStartdt4 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')

        Ann_Cost = arcpy.sa.Con(Ann_Raw > 0, arcpy.sa.Power(Ann_Raw, 1.9), 1)

        # End timing
        processEnd4 = time.time()
        processElapsed4 = int(processEnd4 - processStart4)
        processSuccess_time4 = datetime.datetime.now()
        # Report success
        print(f'p4. Focal Mean Annulus Process completed at {processSuccess_time4.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed4)})')
        print(f'{"*"*100}')

        # Start timing function
        processStart5 = time.time()
        processStartdt5 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')

        #Run focal statistics on the flow accumulation layer to determine the maximum flow accumulation in a 6 cell radius.
        nbr2 = arcpy.sa.NbrCircle(6, "CELL")
        flowmax = arcpy.sa.FocalStatistics(fac_ras, nbr2, "MAXIMUM", "")

        # End timing
        processEnd5 = time.time()
        processElapsed5 = int(processEnd5 - processStart5)
        processSuccess_time5 = datetime.datetime.now()
        # Report success
        print(f'p5. Focal Mean Annulus Process completed at {processSuccess_time5.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed5)})')
        print(f'{"*"*100}')

        # Start timing function
        processStart6 = time.time()
        processStartdt6 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')

        #Apply the hydrologic weights equation copied directly from the STARS tool. In this equation, where the maximum flow accumulation is greater than 1, replace those values with a non-zero topographic wetness index. Where the maximum flow is less than one, or slope is zero, replace those values with 1.
        #This step should take all areas with very low flow accumulation (ie ridgelines) and assign their value as 1. All other areas will have a hydrologic weight calculated.
        flow_wt = arcpy.sa.Con(flowmax > 1, arcpy.sa.Con(fac_ras > 1, (1.0 / flowmax * arcpy.sa.Tan(fac_ras / 57.2957)), 1), 1)

        # End timing
        processEnd6 = time.time()
        processElapsed6 = int(processEnd6 - processStart6)
        processSuccess_time6 = datetime.datetime.now()
        # Report success
        print(f'p.6 Focal Mean Annulus Process completed at {processSuccess_time6.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed6)})')
        print(f'{"*"*100}')

        # Start timing function
        processStart7 = time.time()
        processStartdt7 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')

        #Calculate a cost surface using the flow weight. In this equation, where the DEM cost surface is less than 100 million (ie, all pixels), multiply the cost surface by the flow weight pixels with value greater than or equal to 1. If the flow weight pixels have value less than or equal to 1, multiply the cost surface by 0.1.
        # In areas of high cost identified from the DEM (ie ridgelines), we are multiplying them by the hydrologic weight of 1 (the default hydro weight value for areas of low flow). The high cost on those ridgelines will therefore be unchanged.
        # In all other low cost areas identified from the DEM (ie, non-ridgelines), multiplying them by the hydrologic weight should produce somewhat higher values. (The intention of this tool is a scaling step that will make the following cost allocation tool perform correctly.)
        cost_surf = arcpy.sa.Con(Ann_Cost < 100000000, Ann_Cost * arcpy.sa.Con(flow_wt >= 1, flow_wt, 0.1), 100000000)

        # End timing
        processEnd7 = time.time()
        processElapsed7 = int(processEnd7 - processStart7)
        processSuccess_time7 = datetime.datetime.now()
        # Report success
        print(f'p7. Focal Mean Annulus Process completed at {processSuccess_time7.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed7)})')
        print(f'{"*"*100}')

        # Start timing function
        processStart8 = time.time()
        processStartdt8 = datetime.datetime.now()
        print(f'Begin {datetime.datetime.now()}')

        #Run the Cost Allocation using the rasterized stream reaches and the final cost surface to create the base RCA raster.
        OutRCA = os.path.join(outgdb,prefDict[roi]+'RCA')
        RCA_ras = arcpy.sa.CostAllocation(reachRas, cost_surf)
        #Build a raster attribute table in the RCA raster.
        arcpy.management.BuildRasterAttributeTable(RCA_ras, "Overwrite")
        #Convert the RCA raster to polygon, and dissolve any multipart polygons.
        arcpy.conversion.RasterToPolygon(RCA_ras, "OutRCA_temp", "SIMPLIFY", "Value")
        arcpy.management.Dissolve("OutRCA_temp", OutRCA, "GRIDCODE", "", "MULTI_PART")
        #Add a field for the RCA ID, and calculate that field.
        arcpy.management.AddField(OutRCA, "RCA_ID", "LONG", "", "", "", "")
        arcpy.management.CalculateField(OutRCA, "RCA_ID", "!GRIDCODE!")

        # End timing
        processEnd8 = time.time()
        processElapsed8 = int(processEnd8 - processStart8)
        processSuccess_time8 = datetime.datetime.now()
        # Report success
        print(f'p8. Focal Mean Annulus Process completed at {processSuccess_time8.strftime("%Y-%m-%d %H:%M")} '
              f'(Elapsed time: {datetime.timedelta(seconds=processElapsed8)})')
        print(f'{"*"*100}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')


Begin 2023-11-14 08:22:14.909206
Begin 2023-11-14 08:22:14.910203
Bristol_Bay ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbelev.tif', ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbfac.tif'], ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbfdr.tif'], ['D:\\\\GIS\\\\AKSSF_ValBot_2023\\bbslope.tif'], 'D:\\\\GIS\\\\AKSSF_ValBot_2023\\AKSSF_ValBot.gdb\\Bristol_Bay_streams_merge', 'catID']
Converting D:\\GIS\\AKSSF_ValBot_2023\AKSSF_ValBot.gdb\Bristol_Bay_streams_merge to raster
Stream to raster Process completed at 2023-11-14 08:23 (Elapsed time: 0:01:17)
****************************************************************************************************
Begin 2023-11-14 08:23:32.405811
p3. Focal Mean Annulus Process completed at 2023-11-14 08:23 (Elapsed time: 0:00:00)
****************************************************************************************************
Begin 2023-11-14 08:23:32.529332
p4. Focal Mean Annulus Process completed at 2023-11-14 08:23 (Elapsed time: 0:00:00)
************************************